In [17]:
import random
import string
import numpy as np
from deap import base, creator, tools
from baseball_simulation import test, run_player_scenario, Player

import multiprocessing
pool = multiprocessing.Pool()


In [2]:
test()

0.0
1.0205078125
1.0205078125


In [3]:
class Runner:
    
    def __init__(self, toolbox):
        self.toolbox = toolbox
        self.set_parameters(10, 5, 2)
        
    def set_parameters(self, population_size, iterations, n_matings):
        self.iterations = iterations
        self.population_size = population_size
        self.n_matings = n_matings
        
    def set_fitness(self, population):
        fitnesses = [ 
            (individual, self.toolbox.evaluate(individual)) 
            for individual in population 
        ]

        for individual, fitness in fitnesses:
            individual.fitness.values = (fitness,)
            
    def get_offspring(self, population):
        n = len(population)
        for _ in range(self.n_matings):
            i1, i2 = np.random.choice(range(n), size=2, replace=False)

            offspring1, offspring2 = \
                self.toolbox.mate(population[i1], population[i2])
            
            yield self.toolbox.mutate(offspring1)[0]
            yield self.toolbox.mutate(offspring2)[0]
    
    @staticmethod
    def pull_stats(population, iteration=1):
        fitnesses = [ individual.fitness.values[0] for individual in population ]
        return {
            'i': iteration,
            'mu': np.mean(fitnesses),
            'std': np.std(fitnesses),
            'max': np.max(fitnesses),
            'min': np.min(fitnesses)
        }  
    
    def Run(self):
        population = self.toolbox.population(n=self.population_size)
        self.set_fitness(population)
        
        stats = []
        for iteration in list(range(1, self.iterations + 1)):
            current_population = list(map(self.toolbox.clone, population))
            offspring = list(self.get_offspring(current_population))            
            for child in offspring:
                current_population.append(child)
            
            ## reset fitness,
            self.set_fitness(current_population)
            
            population[:] = self.toolbox.select(current_population, len(population))
            stats.append(
                Runner.pull_stats(population, iteration))
            
        return stats, population

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [5]:
## globals,

random.seed(11);
np.random.seed(121);


INDIVIDUAL_SIZE = NUMBER_OF_CITIES = 10
POPULATION_SIZE = 200
N_ITERATIONS = 100
N_MATINGS = 50

## 

## initial order
people = [5,4,3,2,1,0,9,8,7,6]

In [6]:
toolbox = base.Toolbox()

## permutation setup for individual,
toolbox.register("indices", random.sample, range(INDIVIDUAL_SIZE), INDIVIDUAL_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

## population setup,
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [7]:
def EVALUATE(individual):
    summation = 0
    for i in range(0, len(individual)-1):
        if(individual[i+1] < individual[i]):
            summation += -individual[i+1] + individual[i]
        else:
            summation += -10
    return summation

#toolbox.register("evaluate", EVALUATE)

In [18]:
players =[Player([0.7, 0.3, 0.0, 0.0, 0.0, 0.0]),
          Player([0.5, 0.0, 0.0, 0.0, 0.0, 0.5]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([0.4, 0.0, 0.0, 0.0, 0.0, 0.6]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([0.6, 0.4, 0.0, 0.0, 0.0, 0.0]),
          Player([0.4, 0.0, 0.0, 0.0, 0.0, 0.6]),
          Player([0.8, 0.0, 0.0, 0.0, 0.0, 0.2]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([0.7, 0.3, 0.0, 0.0, 0.0, 0.0])]

players =[Player([1.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
          Player([0.5, 0.0, 0.0, 0.0, 0.0, 0.5]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([0.4, 0.0, 0.0, 0.0, 0.0, 0.6]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([0.6, 0.4, 0.0, 0.0, 0.0, 0.0]),
          Player([0.4, 0.0, 0.0, 0.0, 0.0, 0.6]),
          Player([1.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
          Player([0.5, 0.5, 0.0, 0.0, 0.0, 0.0]),
          Player([1.0, 0.0, 0.0, 0.0, 0.0, 0.0])]

initial_order = [0, 1,2,3,4,5,6,7,8,9]

toolbox.register("evaluate", run_player_scenario, players)
toolbox.register("map", pool.map)

In [19]:
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=10)

In [20]:
a = Runner(toolbox)
a.set_parameters(POPULATION_SIZE, N_ITERATIONS, N_MATINGS)

stats, population = a.Run()

KeyboardInterrupt: 

In [ ]:
fitnesses = sorted([ 
    (i, toolbox.evaluate(individual), individual) 
    for i, individual in enumerate(population) 
], key=lambda x: x[1])

fitnesses[:5]

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)

plt.scatter([ s['min'] for s in stats ], [ s['max'] for s in stats ], marker='*', s=[ (s['std'] + 1) / 20 for s in stats ])

plt.title('min by max')
plt.xlabel('min')
plt.ylabel('max')

plt.plot(stats[0]['min'], stats[0]['max'], marker='.', color='yellow')
plt.plot(stats[-1]['min'], stats[-1]['max'], marker='.', color='red')


plt.subplot(1,2,2)

plt.scatter([ s['i'] for s in stats ], [ s['mu'] for s in stats ], marker='*', s=[ (s['std'] + 1)*10 for s in stats ])

plt.title('average by iteration')
plt.xlabel('iteration')
plt.ylabel('average')

plt.plot(stats[0]['i'], stats[0]['mu'], marker='.', color='yellow')
plt.plot(stats[-1]['i'], stats[-1]['mu'], marker='.', color='red')

plt.tight_layout()
plt.show()

# 